# <font color=darkblue> Machine Learning model deployment with Flask framework on Heroku</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with flask framework on heroku.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')

### 2. Load the dataset

In [2]:
df = pd.read_csv('car+data.csv')

In [3]:
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


### 3. Check the shape and basic information of the dataset.

In [4]:
df.shape

(301, 9)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [6]:
df.duplicated().sum()

2

In [7]:
df.drop_duplicates(inplace=True)
df

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0
...,...,...,...,...,...,...,...,...,...
296,city,2016,9.50,11.60,33988,Diesel,Dealer,Manual,0
297,brio,2015,4.00,5.90,60000,Petrol,Dealer,Manual,0
298,city,2009,3.35,11.00,87934,Petrol,Dealer,Manual,0
299,city,2017,11.50,12.50,9000,Diesel,Dealer,Manual,0


In [9]:
len(df[df.duplicated()])

0

### 5. Drop the columns which you think redundant for the analysis.

In [10]:
df.columns

Index(['Car_Name', 'Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner'],
      dtype='object')

In [11]:
df.drop(['Car_Name'], axis=1, inplace=True)
df.head()

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [12]:
from datetime import date

In [13]:
df['Age_of_the_car'] = date.today().year - df['Year']
df.head()

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,Age_of_the_car
0,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0,9
1,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0,10
2,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0,6
3,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0,12
4,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0,9


In [14]:
df.drop(['Year'], axis=1, inplace=True)

In [15]:
df.head()

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,Age_of_the_car
0,3.35,5.59,27000,Petrol,Dealer,Manual,0,9
1,4.75,9.54,43000,Diesel,Dealer,Manual,0,10
2,7.25,9.85,6900,Petrol,Dealer,Manual,0,6
3,2.85,4.15,5200,Petrol,Dealer,Manual,0,12
4,4.60,6.87,42450,Diesel,Dealer,Manual,0,9


### 7. Encode the categorical columns

In [16]:
df_cat = df.select_dtypes(include='O')
df_cat.head()

,Fuel_Type,Seller_Type,Transmission
0,Petrol,Dealer,Manual
1,Diesel,Dealer,Manual
2,Petrol,Dealer,Manual
3,Petrol,Dealer,Manual
4,Diesel,Dealer,Manual


In [17]:
df['Fuel_Type'].unique()

array(['Petrol', 'Diesel', 'CNG'], dtype=object)

In [19]:
df['Fuel_Type'] = df['Fuel_Type'].replace({'Petrol': 0, 'Diesel':1, 'CNG':2})
df['Fuel_Type'].unique()

array([0, 1, 2], dtype=int64)

In [20]:
df['Seller_Type'].unique()

array(['Dealer', 'Individual'], dtype=object)

In [ ]:
df['Seller_Type'] = df['Seller_Type'].replace({'Dealer':0, 'Individual':1})

In [26]:
df['Seller_Type'].unique()

array([0, 1], dtype=int64)

In [24]:
df['Transmission'].unique()

array(['Manual', 'Automatic'], dtype=object)

In [27]:
df['Transmission'].replace({'Manual': 0, 'Automatic':1}, inplace=True)

In [28]:
df['Transmission'].unique()

array([0, 1], dtype=int64)

### 8. Separate the target and independent features.

In [30]:
## separate target and independent variables
X = df.drop('Selling_Price',axis=1)
y = df['Selling_Price']

### 9. Split the data into train and test.

In [31]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(209, 7) (90, 7)
(209,) (90,)


### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [33]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [36]:
y_test_pred = rf.predict(X_test)
print("Test : ",metrics.r2_score(y_test, y_test_pred))  

0.9002599112778416


In [38]:
y_train_pred = rf.predict(X_train)
print("Train : ", metrics.r2_score(y_train, y_train_pred))

0.9747364091242735


### 11. Create a pickle file with an extension as .pkl

In [40]:
import pickle

In [41]:
pickle.dump(rf, open('model.pkl', 'wb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

In [ ]:
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <meta
      name="viewport"
      content="width=device-width, user-scalable=no, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0"
    />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    <title>Car Price Predictor</title>
  </head>
  <body>
    <div class="login">
      <div id="container">
        <div>
          <h1>Used Car Price Predictor</h1>
          <br />
          <div>
            <h2>{{prediction_text}}</h2>
          </div>
        </div>
      </div>
      <!-- Main Input For Receiving Query to our ML -->
      <form action="{{ url_for('predict')}}" method="post">
        <label for="age">Age of the Car (in Years)</label>
        <br />
        <input
          id="age"
          name="Age_of_the_car"
          placeholder="Age of the Car"
          required="required"
        />
        <br /><br />
        <label for="pres-price">Current Price of the Car (in Lakhs)</label>
        <br />
        <input
          id="pres-price"
          name="Present_Price"
          placeholder="Present Price"
          required="required"
        />
        <br /><br />
        <label for="kms">Driven distance (in kms)</label>
        <br />
        <input
          id="kms"
          name="Kms_Driven"
          placeholder="kms driven"
          required="required"
        />
        <br /><br />
        <label for="owner">Owner Type (1/2/3)</label>
        <br />
        <input
          id="owner"
          type="number"
          name="Owner"
          placeholder="Owner"
          min="1"
          max="3"
          required="required"
        />
        <br /><br />
        <select id="fuel" name="Fuel_Type" required="required">
          <option>Select the Fuel Type</option>
          <option value="0">Petrol</option>
          <option value="1">Diesel</option>
          <option value="2">CNG</option>
        </select>
        <br /><br />

        <select name="Seller_Type" id="seller-type" required="required">
          <option>Seller Type</option>
          <option value="0">Dealer</option>
          <option value="1">Individual</option>
        </select>

        <br /><br />
        <select id="trans" name="Transmission" required="required">
          <option>Transmission Type</option>
          <option value="0">Manual Car</option>
          <option value="1">Automatic Car</option>
        </select>

        <br />
        <div id="sub-btn">
          <button
            type="submit"
            class="btn btn-primary btn-block btn-large"
            id="sub"
          >PREDICT SELLING PRICE
          </button>
        </div>
      </form>

      <br />
      <br />
      {{ prediction_text }}
    </div>
  </body>
</html>

### b) Create app.py file and write the predict function

In [ ]:
from flask import Flask, request, jsonify , render_template
import pickle
import numpy as np
import sklearn

app = Flask(__name__)
model = pickle.load(open('model.pkl','rb'))

@app.route('/', methods=['GET'])
def Homepage():
    return render_template('index.html')
@app.route('/predict', methods =['POST'])
def predict():
    if request.method== 'POST':
        Present_Price = float(request.form['Present_Price'])
        Kms_Driven = int(request.form['Kms_Driven'])
        Owner = int(request.form['Owner'])
        Fuel_Type = request.form['Fuel_Type']
        Age_of_the_car = request.form['Age_of_the_car']
        Seller_Type = request.form['Seller_Type']
        Transmission = request.form['Transmission']

        prediction = model.predict([[Present_Price, Kms_Driven, Owner, Fuel_Type, Age_of_the_car, Seller_Type, Transmission]])
        output = round(prediction[0],2)
        return render_template('index.html', prediction_text = "You can sell your car at {} lakhs".format(output))




# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    app.run(debug=True)

### Happy Learning :)